In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mesothelioma/GSE163720'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Association of RERG Expression to Female Survival Advantage in Malignant Pleural Mesothelioma I"
!Series_summary	"Sex differences in incidence, prognosis, and treatment response have been described for many cancers. In malignant pleural mesothelioma (MPM), a lethal disease associated with asbestos exposure, men outnumber women 4 to 1, but women consistently live longer than men following surgery-based therapy. This study investigated whether tumor expression of genes associated with estrogen signaling could potentially explain observed survival differences. Two microarray datasets of MPM tumors were analyzed to discover estrogen-related genes associated with survival. A validation cohort of MPM tumors was selected to balance the numbers of men and women and control for competing prognostic influences. The RAS like estrogen regulated growth inhibitor (RERG) gene was identified as the most differentially-expressed estrogen-related gene in these tumo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True

# Define keys for trait, age, and gender rows based on sample characteristics
trait_row = 0  # for Mesothelioma
age_row = None  # No mention of age-related data in the characteristics
gender_row = 2  # for Sex

# Data type conversion functions
def convert_trait(value):
    # Extract the value after the colon and convert it to binary (1 for presence, 0 for absence)
    try:
        val = value.split(":")[1].strip().lower()
        if val in ['yes', 'present', 'tumor']:  # Assuming Mesothelioma presence is indicated in such a way
            return 1
        elif val in ['no', 'absent']:
            return 0
    except IndexError:
        return None

def convert_age(value):
    # Age data is not available in this dataset
    return None

def convert_gender(value):
    # Extract the value after the colon and convert it to binary (0 for female, 1 for male)
    try:
        val = value.split(":")[1].strip().upper()
        if val == 'F':
            return 0
        elif val == 'M':
            return 1
    except IndexError:
        return None

# Save cohort information
save_cohort_info('GSE163720', './preprocessed/Mesothelioma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Mesothelioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Mesothelioma/trait_data/GSE163720.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4984371': [1, 0], 'GSM4984372': [1, 0], 'GSM4984373': [1, 0], 'GSM4984374': [1, 1], 'GSM4984375': [1, 0], 'GSM4984376': [1, 0], 'GSM4984377': [1, 0], 'GSM4984378': [1, 1], 'GSM4984379': [1, 0], 'GSM4984380': [1, 0], 'GSM4984381': [1, 1], 'GSM4984382': [1, 0], 'GSM4984383': [1, 0], 'GSM4984384': [1, 0], 'GSM4984385': [1, 0], 'GSM4984386': [1, 1], 'GSM4984387': [1, 0], 'GSM4984388': [1, 0], 'GSM4984389': [1, 0], 'GSM4984390': [1, 0], 'GSM4984391': [1, 1], 'GSM4984392': [1, 1], 'GSM4984393': [1, 0], 'GSM4984394': [1, 1], 'GSM4984395': [1, 1], 'GSM4984396': [1, 1], 'GSM4984397': [1, 0], 'GSM4984398': [1, 0], 'GSM4984399': [1, 0], 'GSM4984400': [1, 0], 'GSM4984401': [1, 1], 'GSM4984402': [1, 0], 'GSM4984403': [1, 0], 'GSM4984404': [1, 0], 'GSM4984405': [1, 0], 'GSM4984406': [1, 0], 'GSM4984407': [1, 0], 'GSM4984408': [1, 1], 'GSM4984409': [1, 1], 'GSM4984410': [1, 0], 'GSM4984411': [1, 0], 'GSM4984412': [1, 0], 'GSM4984413': [1, 0], 'GSM4984414': [1, 0], 'GSM4984415': [1, 0], 'GSM49844

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['7892501', '7892502', '7892503', '7892504', '7892505', '7892506',
       '7892507', '7892508', '7892509', '7892510', '7892511', '7892512',
       '7892513', '7892514', '7892515', '7892516', '7892517', '7892518',
       '7892519', '7892520'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. When analyzing a gene expression dataset, we need to map some identifiers of genes to actual gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'gene_assignment'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
